In [ ]:
#default_exp data

In [ ]:
#export
from fastai2.basics import *

In [ ]:
#export
class IncrementalSplitter:
  'Dynamically changes pct of data used without mixing train and val data'
  @delegates(RandomSplitter)
  def __init__(self, pct=1., **kwargs):
    self.pct = pct
    seed = kwargs.pop('seed', None) or random.randint(0,1e6)
    self._splitter = RandomSplitter(seed=seed, **kwargs)
    
  def __call__(self, o, **kwargs):
    tidxs, vidxs = self._splitter(o, **kwargs)
    random.shuffle(tidxs); random.shuffle(vidxs)
    return tidxs[:int(len(tidxs)*self.pct)], vidxs[:int(len(vidxs)*self.pct)]

In [ ]:
items = np.arange(1000)
splitter = IncrementalSplitter()
t1,v1 = splitter(items)
t2,v2 = splitter(items)
test_eq(set(t1), set(t2))
test_eq(set(v1), set(v2))

### Calculate coco variance

In [ ]:
# coco = untar_data(URLs.COCO_SAMPLE)
# dset = Datasets(get_image_files(coco), tfms=[PILImage.create])
# dl = dset.dataloaders(after_item=[ToTensor(), Resize(512)], after_batch=[IntToFloatTensor()])
# c_mean = partial(torch.mean, axis=[0,2,3])
# c_std = partial(torch.std, axis=[0,2,3])
# means,stds = zip(*[(c_mean(x[0]),c_std(x[0])) for x in progress_bar(dl.train)])
# mean = torch.stack(means).mean(axis=0)
# std  = torch.stack(stds).pow(2).mean(axis=0).sqrt()

In [ ]:
#export
coco_stats = ([0.4605, 0.4101, 0.3642], [0.2780, 0.2701, 0.2741])

## Export - 

In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted 01_data.ipynb.
Converted 02_loss.ipynb.
Converted 03_models.ipynb.
Converted Untitled.ipynb.
Converted Untitled1.ipynb.
